# Westeros Tutorial
## Adding additional technology functionalities: Introducing `addon` technologies

This tutorial is concerned with is looking at how to add additional features, known as `addon` technologies such as carbon-capture-and-storage (CCS) retrofits, passout-turbines (for optional heat cogeneration) or cooling technologies, to existing technologies.

There are several ways in which to tackle this issue.  Lets take for example our `coal_ppl`.  All of the above mentioned additional features could be implemented by introducing different modes for the `coal_ppl`.  For example, heat cogeneration could be implemented as a separate `mode` of the `coal_ppl`, where instead of just generating electricity, heat can also be produced at the cost of reducing the amount of electricity generated. Another approach would make use of the generic `relations`, therefore linking the a newly added technology representing the passout-turbine with the activity of the `coal_ppl`.  Both of these approaches have some downsides. Using a separate `mode` will not permit explicitly modelling investment costs and lifetimes associated with the asset being added to the `coal_ppl`.  Generic relations are very flexible, but if too many of these exist, then the modell quickly becomes very hard to understand.  Therefore, MESSAGEix offers an explicit `addon` formulation.

What this means, is that the additional technology options are explicitly modelled as separate technologies, classified as `addon` technologies, linked to the activity of the technology to which they serve as additional configuration options, the parent technology, through specifying a `addon_conversion` factor.  The activity of the `addon` technology can further be restricted to a minimum or maximum share of the activity of the parent technology.

## Adding heat co-generation capacibility to the `coal_ppl`
This tutorial will extend the current reference-energy-system to include a demand for heat and the necessary technologies to meet this demand.  Heat will be generated via a `passout-turbine` which will be linked to the `coal_ppl` using the `addon` formulation.

# *Add modified RES*

In [ ]:
import pandas as pd
import ixmp
import message_ix

from message_ix.utils import make_df

%matplotlib inline

In [ ]:
mp = ixmp.Platform()

## Load the existing scenario '*baseline*' and clone to a new scenario '*addon_technology*' which we will extended to include heat cogeneration using the `addon` feature

In [ ]:
model = 'Westeros Electrified'
base = message_ix.Scenario(mp, model=model, scenario='baseline')
scen = base.clone(model, 'addon_technology', 'illustration of addon formulation', keep_solution=False)
scen.check_out()

## Retrieve parameters to perform subsequent addition of parameters

In [ ]:
year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen.set('year')
country = 'Westeros'
gdp_profile = pd.Series([1., 1.5, 1.9], index=pd.Index([700, 710, 720], name='Time'))

## Define helper dataframes used for subsequent operations

In [ ]:
base_input = {
    'node_loc': country,
    'year_vtg': vintage_years
    ,
    'year_act': act_years,
    'mode': 'standard',
    'node_origin': country,
    'commodity': 'electricity',
    'time': 'year',
    'time_origin': 'year',
}

base_output = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'node_dest': country,
    'time': 'year',
    'time_dest': 'year', 
    'unit': '%',
}
base_capacity_factor = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'time': 'year',
    'unit': '%',
}
base_technical_lifetime = {
    'node_loc': country,
    'year_vtg': model_horizon,
    'unit': 'y',
}
base_inv_cost = {
    'node_loc': country,
    'year_vtg': model_horizon,
    'unit': 'USD/GWa',
}
base_fix_cost = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'unit': 'USD/GWa',
}
base_var_cost = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'time': 'year',
    'unit': 'USD/GWa',
}

## `addon` technology - interlinking technologies
In order to demonstrate the use of this feature, we will be adding a new demand for heat to this scenario.  This demand will be met by the `coal_ppl` via a passout-turbine, the `addon` technology.

We will therefore go through the following steps:
1. Define a new demand for heat.
2. Add new technologies: passout-turbine, district heat network, in-house district heat connection
3. Link the passout-turbine to the coal_ppl using the `addon` feature.

### Step1: Define a new demand
Note: Austria has an approximate space heating requirement of about 47000 GWh supplied via district heat, at approximately 9 Mio. inhabitants this equates to approximately 5000 kWh per capita.

In [ ]:
# Define a new commodity `heat`
scen.add_set("commodity", ["heat"])

# Add heat demand at the useful level
heat_demand = pd.DataFrame({
        'node': country,
        'commodity': 'heat',
        'level': 'useful',
        'year': [700, 710, 720],
        'time': 'year',
        'value': (50 * gdp_profile).round(),
        'unit': 'GWa',
    })
scen.add_par("demand", heat_demand)

### Step2: Define new technologies inlcuding a passout-turbine along with a grid and an in-house heat-distribution system

Passout-turbine (`po_turbine`) characteristics: The passout-turbine requires one unit of electricity to generate five units of heat. The lifetime is assumed to be 30 years, 10 years longer then that of the `coal_ppl`.  Investment costs are 150\$/kW compared to 500\$/kW for the `coal_ppl`.  A coal heatplant would have higher investment costs, approximately double that of the `po_turbine`.

District heat (`dh_grid`) network characteristics: District heating networks have only very low losses as these cover only short distances (within city perimeters). We will assume the district heating network to have an efficiency of 97%.

As before we will work our way backwards, starting from the `heat` demand defined at the `useful` energy level and connecting this to the `final` energy level via a technology, `dh_house`, representing the in-house heat distribution system.

In [ ]:
tec = 'dh_house'
scen.add_set('technology', tec)

dh_house_out = make_df(base_output, technology=tec, commodity='heat', 
                   level='useful', value=1.0)
scen.add_par('output', dh_house_out)

dh_house_in = make_df(base_input, technology=tec, commodity='heat',  
                  level='final', value=1.0, unit='%')
scen.add_par('input', dh_house_in)

Next, we will add the district heating network

In [ ]:
tec = 'dh_grid'
scen.add_set('technology', tec)

dh_grid_out = make_df(base_output, technology='dh_grid', commodity='heat', 
                   level='final', value=1.0)
scen.add_par('output', dh_grid_out)

dh_grid_in = make_df(base_input, technology='dh_grid', commodity='heat',  
                  level='secondary', value=1.03, unit='%')
scen.add_par('input', dh_grid_in)

The last technology we will add is the `po_turbine` itself.

In [ ]:
tec = 'po_turbine'
scen.add_set('technology', tec)

po_out = make_df(base_output, technology=tec, commodity='heat', 
                   level='secondary', value=1.0)
scen.add_par('output', po_out)

po_in = make_df(base_input, technology=tec, commodity='electricity',  
                  level='secondary', value=0.2, unit='%')
scen.add_par('input', po_in)

po_tl = make_df(base_technical_lifetime, technology=tec, value=30)
scen.add_par('technical_lifetime', po_tl)

po_inv = make_df(base_inv_cost, technology=tec, value=150)
scen.add_par('inv_cost', po_inv)

po_fix = make_df(base_fix_cost, technology=tec, value=15)
scen.add_par('fix_cost', po_fix)

### Step3: Link the `po_turbine` with `coal_ppl`
The `po_turbine` could already operate as all required parameters are defined, yet without a link to the activity of the `coal_ppl`, the `po_turbine` has the possibility of using electricity generated from either the `coal_ppl` or  the `wind_ppl`.  But because the `po_turbine` is an addon component to the `coal_ppl` a distinct linkage needs to be established.

First, the newly added technology `po_turbine` needs to be classified as an `addon` technology

In [ ]:
scen.add_set('addon', 'po_turbine')

Next, we need a new `type_addon`, which we will name `cogeneration_heat`.  We will classify the `po_turbine` via the *category* `addon` as one of the addon technologies as part of this specific `type_addon`.  In some cases, for example when modelling cooling technologies, multiple technologies can be classfied within a single `type_addon`.

Via the set `map_tec_addon` we map the electricity generation technology, `coal_ppl`, to the `addon` technology, `po_turbine`. Multiple technologies, for example further fossil powerplants, could also be added to the this `type_addon` so as to be able to produce heat via the `po_turbine`.

Note: the `addon` technology as well as the parent technology must have the same `mode`.

In [ ]:
type_addon = 'cogeneration_heat'
addon = 'po_turbine'
tec = 'coal_ppl'
scen.add_cat('addon', type_addon, addon)
scen.add_set('map_tec_addon', pd.DataFrame({'technology': tec,
                                            'type_addon': [type_addon]}))

The last step required in order to link the `coal_ppl` is to define the `addon_conversion` factor between the `coal_ppl` and the `type_addon`.  This is important, because the `coal_ppl` generates electricty while the `po_turbine` generates heat. Therefore, we can use the inverse of the `input` coefficient from the `po_turbine`.

In [ ]:
# Index for `addon_conversion` is ['node', 'technology', 'year_vtg',
#                                  'year_act', 'mode', 'time',
#                                  'type_addon', 'value', 'unit']
df = pd.DataFrame({'node': country,
                   'technology': tec,
                   'year_vtg': vintage_years,
                   'year_act': act_years,
                   'mode': 'standard',
                   'time': 'year',
                   'type_addon': type_addon,
                   'value': 5,
                   'unit': '%'})
scen.add_par('addon_conversion', df)

Although not necessary for this specific example, it is also possible to limit the activity of the `po_turbine` to a specific share of the `coal_ppl` activity. In the example below, the `po_turbine` is limited to using 15% of the `coal_ppl` activity.  Likewise, a constraint on the minimum amount of electricity used from the `po_turbine` can be applied by using the parameter `addon_lo`.

In [ ]:
# Index for `addon_up` is ['node', 'technology', 'year_act',
#                          'mode', 'time', 'type_addon',
#                          'value', 'unit']
df = pd.DataFrame({'node': country,
                   'technology': tec,
                   'year_act': act_years,
                   'mode': 'standard',
                   'time': 'year',
                   'type_addon': type_addon,
                   'value': .15,
                   'unit': '%'})
scen.add_par('addon_up', df)

### Commit and solve

In [ ]:
scen.commit(comment='define parameters for renewable implementation')
scen.set_as_default()

In [ ]:
scen.solve()

In [ ]:
scen.var('OBJ')['lvl']

# Plotting Results

In [ ]:
from tools import Plots
p = Plots(scen, country, firstyear=700) # scenario: 'renewable_potential' (emission_bound scenario with firm capacity)
b = Plots(base, country, firstyear=700) # scenario: 'carbon_tax' (without renewable potentials)

## Activity
***
In the new scenario ('*renewable_potential*'), the effects of the addon technology can be seen when comparing the activity to the baseline scenario ('*baseline*'). From 700 onwards, the activity of the `wind_ppl` has increased to compensate for the electricity required from the `coal_ppl` for use in the `po_turbine`. In 720, when the `wind_ppl` is phased out, then more electricity is required to be produced by the `coal_ppl`. 

### Scenario: '*carbon_tax*'

In [ ]:
b.plot_activity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

### Scenario: '*renewable_potential*'

In [ ]:
p.plot_activity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

## Capacity
***
The behavior observed for the activity of the two electricity generation technologies is reflected in the capacity.

### Scenario: '*carbon_tax*'

In [ ]:
b.plot_capacity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

### Scenario: '*renewable_potential*'

In [ ]:
p.plot_capacity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

## Prices
***
The resulting impact on the electricity price though is negligable.

### Scenario: '*carbon_tax*'

In [ ]:
b.plot_prices(subset=['light'], baseyear=True)

### Scenario: '*renewable_potential*'

In [ ]:
p.plot_prices(subset=['light'], baseyear=True)

In [ ]:
mp.close_db()